# Import Library

In [1]:
import numpy as np
import pandas as pd
import json

import matplotlib.pyplot as plt 
from matplotlib.lines import Line2D
import seaborn as sns

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import BallTree
from scipy.spatial import KDTree

In [2]:
pd.options.display.max_columns = None
pd.options.display.float_format = '{:.2f}'.format
pd.reset_option('display.max_rows', None)
pd.reset_option('display.max_columns', None)
pd.reset_option('display.max_colwidth', None)

# Import Data

## DKI Jakarta

In [3]:
dki_paths = [
    #r'.\dataset\DKI Jakarta\Jakarta_details.csv',
    r'.\dataset\DKI Jakarta\Revisi Jakarta 1\Jakarta_details_rev1.csv',
    r'.\dataset\DKI Jakarta\Revisi Jakarta 2\Jakarta_details_rev2.csv',
    r'.\dataset\DKI Jakarta\Revisi Jakarta 3\Jakarta_details_rev3.csv'
]

In [4]:
dki_dfs = [pd.read_csv(dki_path) for dki_path in dki_paths]
dki_xlsx = pd.read_excel('.\dataset\DKI Jakarta\Jakarta_details.xlsx')

## Banten

In [5]:
banten_paths = [
    r'.\dataset\Banten\Banten_details.csv',
    r'.\dataset\Banten\Revisi Banten 1\Banten_details_rev1.csv',
    r'.\dataset\Banten\Revisi Banten 2\Banten_details_rev2.csv',
    r'.\dataset\Banten\Revisi Banten 3\Banten_details_rev3.csv',
    r'.\dataset\Banten\Revisi Banten 4\Banten_details_rev4.csv',
    r'.\dataset\Banten\Revisi Banten 5\Banten_details_rev5.csv'
]

In [6]:
banten_dfs = [pd.read_csv(banten_path) for banten_path in banten_paths]

## Jawa Barat

In [7]:
jabar_paths = [
    r'.\dataset\Jawa Barat\Jabar_details.csv',
    r'.\dataset\Jawa Barat\Revisi Jawa Barat 1\Jabar_details_rev1.csv',
    r'.\dataset\Jawa Barat\Revisi Jawa Barat 2\Jabar_details_rev2.csv',
    r'.\dataset\Jawa Barat\Revisi Jawa Barat 3\Jabar_details_rev3.csv',
    r'.\dataset\Jawa Barat\Revisi Jawa Barat 4\Jabar_details_rev4.csv'
]

In [8]:
jabar_dfs = [pd.read_csv(jabar_path) for jabar_path in jabar_paths]

## Jawa Tengah

In [9]:
jateng_paths = [
    #r'.\dataset\DKI Jakarta\Jakarta_details.csv',
    r'.\dataset\Jawa Tengah\Revisi Jawa Tengah 1\jateng_details_rev1.csv',
    r'.\dataset\Jawa Tengah\Revisi Jawa Tengah 2\jateng_details_rev2.csv',
    r'.\dataset\Jawa Tengah\Revisi Jawa Tengah 3\jateng_details_rev3.csv'
]

In [10]:
jateng_dfs = [pd.read_csv(jateng_path) for jateng_path in jateng_paths]
jateng_xlsx = pd.read_excel('.\dataset\Jawa Tengah\Jateng_details.xlsx')

## Jawa Timur

In [11]:
jatim_paths = [
    r'.\dataset\Jawa Timur\jatim_details.csv',
    r'.\dataset\Jawa Timur\Revisi Jawa Timur 1\jatim_details_rev1.csv',
    r'.\dataset\Jawa Timur\Revisi Jawa Timur 2\jatim_details_rev2.csv',
    r'.\dataset\Jawa Timur\Revisi Jawa Timur 3\jatim_details_rev3.csv'
]

In [12]:
jatim_dfs = [pd.read_csv(jatim_path) for jatim_path in jatim_paths]

## DIY

In [13]:
diy_paths = [
    r'.\dataset\DIY\DIY_details.csv',
    r'.\dataset\DIY\Revisi DIY 1\DIY_details_rev1.csv',
    r'.\dataset\DIY\Revisi DIY 2\DIY_details_rev2.csv',
    r'.\dataset\DIY\Revisi DIY 3\DIY_details_rev3.csv'
]

In [14]:
diy_dfs = [pd.read_csv(diy_path) for diy_path in diy_paths]

# Processing Clean Data

## Combining

In [15]:
merged_dki = pd.concat([pd.concat(dki_dfs, ignore_index=True), dki_xlsx], ignore_index=True)
merged_banten = pd.concat(banten_dfs, ignore_index=True)
merged_jabar = pd.concat(jabar_dfs, ignore_index=True)
merged_jateng = pd.concat([pd.concat(jateng_dfs, ignore_index=True), jateng_xlsx], ignore_index=True)
merged_jatim = pd.concat(jatim_dfs, ignore_index=True)
merged_diy = pd.concat(diy_dfs, ignore_index=True)

## Remove Duplicate

In [16]:
df_dki = merged_dki.drop_duplicates(subset='place_id')
df_banten = merged_banten.drop_duplicates(subset='place_id')
df_jabar = merged_jabar.drop_duplicates(subset='place_id')
df_jateng = merged_jateng.drop_duplicates(subset='place_id')
df_jatim = merged_jatim.drop_duplicates(subset='place_id')
df_diy = merged_diy.drop_duplicates(subset='place_id')

## Drop Features

In [17]:
df_diy.columns

Index(['place_id', 'name', 'description', 'is_spending_on_ads', 'reviews',
       'competitors', 'website', 'can_claim', 'emails', 'phones', 'linkedin',
       'twitter', 'facebook', 'youtube', 'instagram', 'pinterest', 'github',
       'snapchat', 'tiktok', 'owner', 'featured_image', 'main_category',
       'categories', 'rating', 'workday_timing', 'closed_on', 'phone',
       'address', 'review_keywords', 'link', 'status', 'price_range',
       'reviews_per_rating', 'featured_question', 'reviews_link',
       'coordinates', 'plus_code', 'detailed_address', 'time_zone', 'cid',
       'data_id', 'about', 'images', 'hours', 'most_popular_times',
       'popular_times', 'menu', 'reservations', 'order_online_links',
       'featured_reviews', 'detailed_reviews'],
      dtype='object')

In [18]:
selected_col_clean = ['place_id', 'name', 'address', 'featured_image',
                      'reviews', 'main_category', 'categories', 'rating','reviews_per_rating',
                      'coordinates', 'most_popular_times'
                     ]

In [19]:
clean_df_dki = df_dki[selected_col_clean]
clean_df_banten = df_banten[selected_col_clean]
clean_df_jabar = df_jabar[selected_col_clean]
clean_df_jateng = df_jateng[selected_col_clean]
clean_df_jatim = df_jatim[selected_col_clean]
clean_df_diy = df_diy[selected_col_clean]

In [20]:
full_df = pd.concat([clean_df_dki, clean_df_banten, clean_df_jabar, clean_df_jateng, clean_df_jatim, clean_df_diy], ignore_index=True)
full_df = full_df.drop_duplicates(subset='place_id', ignore_index=True)

In [21]:
full_df = full_df.dropna(subset='place_id', ignore_index=True)

In [22]:
# Export for Clustering

full_df.to_csv('./clustering_dataset/raw_cluster.csv')
clean_df_dki.to_csv('./clustering_dataset/raw_dki_cluster.csv') 
clean_df_banten.to_csv('./clustering_dataset/raw_banten_cluster.csv') 
clean_df_jabar.to_csv('./clustering_dataset/raw_jabar_cluster.csv') 
clean_df_jateng.to_csv('./clustering_dataset/raw_jateng_cluster.csv') 
clean_df_jatim.to_csv('./clustering_dataset/raw_jatim_cluster.csv') 
clean_df_diy.to_csv('./clustering_dataset/raw_diy_cluster.csv') 

## Feature Engineer

### Fill NaN with None

In [23]:
# # Daftar kolom yang akan diisi nilai 'None'
# columns_to_fill = ['name', 'address', 'featured_image']

# # Mengganti nilai yang hilang dengan 'None' pada kolom yang ditentukan
# full_df[columns_to_fill] = full_df[columns_to_fill].fillna('None')

### Reviews per Rating replace null

In [24]:
# Fungsi untuk mengganti null dengan 0
def replace_null_with_zero(json_str):
    # Mengkonversi string JSON ke dictionary
    json_dict = json.loads(json_str)
    
    # Mengganti nilai null dengan 0
    for key, value in json_dict.items():
        if value is None:
            json_dict[key] = 0
    
    # Mengembalikan dictionary sebagai string JSON
    return json.dumps(json_dict)

# Menerapkan fungsi ke kolom 'reviews_per_rating'
full_df['reviews_per_rating'] = full_df['reviews_per_rating'].apply(replace_null_with_zero)

In [25]:
full_df

,place_id,name,address,featured_image,reviews,main_category,categories,rating,reviews_per_rating,coordinates,most_popular_times
0,ChIJd0EvQ4_zaS4ROHLtujX-hmE,Bebek Kaleyo Tebet,"Lapangan Ros Selatan No.49, RT.4/RW.1, Tebet T...",https://lh5.googleusercontent.com/p/AF1QipO_gO...,11016,Restoran Indonesia,"[""Restoran Indonesia"", ""Restoran Asia""]",4.50,"{""1"": 184, ""2"": 152, ""3"": 791, ""4"": 2528, ""5"":...","{""latitude"": -6.2258, ""longitude"": 106.8568683}","[{""hour_of_day"": 19, ""average_popularity"": 80...."
1,ChIJEwwrrXH3aS4RcBo0XDRfOnc,McDonald's Puri Kembangan,"Jl. Puri Indah Raya Blok U1 No.1, RT.3/RW.2, K...",https://lh5.googleusercontent.com/p/AF1QipNF6A...,9072,Restoran Cepat Saji,"[""Restoran Cepat Saji""]",4.60,"{""1"": 204, ""2"": 117, ""3"": 490, ""4"": 1760, ""5"":...","{""latitude"": -6.187604599999999, ""longitude"": ...","[{""hour_of_day"": 23, ""average_popularity"": 88...."
2,ChIJC_lABffzaS4RZGzb2lD-iAw,Setiabudi One,"Jl. H. R. Rasuna Said No.Kav.62, Kuningan, Kar...",https://lh5.googleusercontent.com/p/AF1QipMPXl...,8456,Pujasera,"[""Pujasera""]",4.60,"{""1"": 61, ""2"": 48, ""3"": 529, ""4"": 1996, ""5"": 5...","{""latitude"": -6.2152515, ""longitude"": 106.8299...","[{""hour_of_day"": 18, ""average_popularity"": 84...."
3,ChIJwZMcGHXxaS4RKdlMwHZXitw,KFC Gunawarman,"No.80 Blok Q1, Jl. Gunawarman, Rw. Bar., Kec. ...",https://lh5.googleusercontent.com/p/AF1QipNYUw...,5918,Restoran Cepat Saji,"[""Restoran Cepat Saji"", ""Kafe"", ""Restoran Masa...",4.50,"{""1"": 165, ""2"": 76, ""3"": 465, ""4"": 1300, ""5"": ...","{""latitude"": -6.238975, ""longitude"": 106.8074383}","[{""hour_of_day"": 8, ""average_popularity"": 74.8..."
4,ChIJp5VZTgT0aS4R26oBEPT4nUw,Warung MJS,"Jl. Setia Budi Tengah No.11, RT.1/RW.1, Kuning...",https://lh5.googleusercontent.com/p/AF1QipO6TD...,5896,Restoran Jawa,"[""Restoran Jawa""]",4.50,"{""1"": 76, ""2"": 63, ""3"": 385, ""4"": 1478, ""5"": 3...","{""latitude"": -6.2087667, ""longitude"": 106.8267...","[{""hour_of_day"": 13, ""average_popularity"": 91...."
...,...,...,...,...,...,...,...,...,...,...,...
30184,ChIJG3hFmuRZei4Ro7gGAxbNeoY,Yongky 1,"696G+C4G Jogja dekat rel, Bausasran, Kec. Danu...",https://streetviewpixels-pa.googleapis.com/v1/...,0,Kedai Kopi,"[""Kedai Kopi""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -7.7889428999999994, ""longitude"":...",Not Present
30185,ChIJF2FHPgBXei4R1LyeLo47Ppc,angkringan sekh wanto,"Jl. Tegal Panggung No.4, Tegal Panggung, Kec. ...",https://streetviewpixels-pa.googleapis.com/v1/...,0,Kedai Kopi,"[""Kedai Kopi""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -7.7966305, ""longitude"": 110.3714...",Not Present
30186,ChIJh2ZkJ34xEm4R7QzIh_u7Gg4,catering,NaN,NaN,0,Toko Makanan Sehat,"[""Toko Makanan Sehat"", ""Service establishment""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -2.44565, ""longitude"": 117.888799...",Not Present
30187,ChIJ0V0zLK9Zei4RUUy9oy4BRa8,yogyakarta,"6939+QQ9, Suryatmajan, Kec. Danurejan, Kota Yo...",https://streetviewpixels-pa.googleapis.com/v1/...,0,Toko bahan makanan,"[""Toko bahan makanan""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -7.7955798, ""longitude"": 110.3694...",Not Present


### Replace null in 'main_category' and 'categories'

In [26]:
full_df['main_category'] = full_df['main_category'].fillna(value='Restoran')
full_df['categories'] = full_df['categories'].fillna(value='["Restoran"]')

In [27]:
full_df.isna().sum()

place_id                0
name                   10
address                84
featured_image        862
reviews                 0
main_category           0
categories              0
rating                  0
reviews_per_rating      0
coordinates             0
most_popular_times      0
dtype: int64

### Main_Category Cardinality

In [28]:
# Menghitung jumlah nilai unik dalam kolom 'main_category'
num_unique_values = full_df['main_category'].nunique()
print(f"Jumlah nilai unik dalam 'main_category': {num_unique_values}")

# Mendapatkan daftar nilai unik dalam kolom 'main_category'
unique_values = full_df['main_category'].unique()
print(f"Nilai unik dalam 'main_category': {unique_values}")


Jumlah nilai unik dalam 'main_category': 345
Nilai unik dalam 'main_category': ['Restoran Indonesia' 'Restoran Cepat Saji' 'Pujasera' 'Restoran Jawa'
 'Restoran Masakan Barat' 'Tempat Minum Bir Luar Ruangan' 'Kafe'
 'Kedai Kopi' 'Restoran Steak' 'Restoran' 'Restoran Seafood'
 'Restoran Ramen' 'Bar Dengan Pentas Musik' 'Bar' 'Restoran India'
 'Restoran Jerman' 'Restoran Padang' 'Restoran Spanyol' 'Restoran Italia'
 'Restoran Thailand' 'Restoran Bali' 'Toko Bubble Tea' 'Restoran Manado'
 'Restoran Timur Tengah' 'Restoran Sunda' 'Restoran Jepang'
 'Restoran Masakan Ayam' 'Lounge' 'Restoran Korea' 'Kios Sandwich'
 'Restoran Pizza' 'Restoran China' 'Restoran Mie' 'Restoran Vegan'
 'Restoran Asia' 'Restoran Nasi Uduk' 'Restoran Makanan Sehat'
 'Restoran Betawi' 'Toko Swalayan' 'Toko Roti' 'Bar & Grill' 'Bar Hokah'
 'Toko Tahu' 'Pub' 'Restoran Singapura' 'Restoran Keluarga' 'Bistro'
 'Restoran Yakiniku' 'Gedung Pernikahan' 'Restoran Pempek'
 'Bar Kopi Espresso' 'Kafe Coklat' 'Kedai Jus' 'Bar 

In [29]:
# Membaca dictionary pengelompokan dari file .txt
with open('category_groups.txt', 'r', encoding='utf-8') as file:
    category_groups = json.load(file)

# Fungsi untuk mengelompokkan kategori
def categorize(category):
    for group, categories in category_groups.items():
        if category in categories:
            return group
    return "Lainnya"

# Membuat kolom baru 'main_category_group' di DataFrame
full_df['main_category'] = full_df['main_category'].apply(categorize)

In [30]:
full_df

,place_id,name,address,featured_image,reviews,main_category,categories,rating,reviews_per_rating,coordinates,most_popular_times
0,ChIJd0EvQ4_zaS4ROHLtujX-hmE,Bebek Kaleyo Tebet,"Lapangan Ros Selatan No.49, RT.4/RW.1, Tebet T...",https://lh5.googleusercontent.com/p/AF1QipO_gO...,11016,Restoran,"[""Restoran Indonesia"", ""Restoran Asia""]",4.50,"{""1"": 184, ""2"": 152, ""3"": 791, ""4"": 2528, ""5"":...","{""latitude"": -6.2258, ""longitude"": 106.8568683}","[{""hour_of_day"": 19, ""average_popularity"": 80...."
1,ChIJEwwrrXH3aS4RcBo0XDRfOnc,McDonald's Puri Kembangan,"Jl. Puri Indah Raya Blok U1 No.1, RT.3/RW.2, K...",https://lh5.googleusercontent.com/p/AF1QipNF6A...,9072,Restoran,"[""Restoran Cepat Saji""]",4.60,"{""1"": 204, ""2"": 117, ""3"": 490, ""4"": 1760, ""5"":...","{""latitude"": -6.187604599999999, ""longitude"": ...","[{""hour_of_day"": 23, ""average_popularity"": 88...."
2,ChIJC_lABffzaS4RZGzb2lD-iAw,Setiabudi One,"Jl. H. R. Rasuna Said No.Kav.62, Kuningan, Kar...",https://lh5.googleusercontent.com/p/AF1QipMPXl...,8456,Restoran,"[""Pujasera""]",4.60,"{""1"": 61, ""2"": 48, ""3"": 529, ""4"": 1996, ""5"": 5...","{""latitude"": -6.2152515, ""longitude"": 106.8299...","[{""hour_of_day"": 18, ""average_popularity"": 84...."
3,ChIJwZMcGHXxaS4RKdlMwHZXitw,KFC Gunawarman,"No.80 Blok Q1, Jl. Gunawarman, Rw. Bar., Kec. ...",https://lh5.googleusercontent.com/p/AF1QipNYUw...,5918,Restoran,"[""Restoran Cepat Saji"", ""Kafe"", ""Restoran Masa...",4.50,"{""1"": 165, ""2"": 76, ""3"": 465, ""4"": 1300, ""5"": ...","{""latitude"": -6.238975, ""longitude"": 106.8074383}","[{""hour_of_day"": 8, ""average_popularity"": 74.8..."
4,ChIJp5VZTgT0aS4R26oBEPT4nUw,Warung MJS,"Jl. Setia Budi Tengah No.11, RT.1/RW.1, Kuning...",https://lh5.googleusercontent.com/p/AF1QipO6TD...,5896,Restoran,"[""Restoran Jawa""]",4.50,"{""1"": 76, ""2"": 63, ""3"": 385, ""4"": 1478, ""5"": 3...","{""latitude"": -6.2087667, ""longitude"": 106.8267...","[{""hour_of_day"": 13, ""average_popularity"": 91...."
...,...,...,...,...,...,...,...,...,...,...,...
30184,ChIJG3hFmuRZei4Ro7gGAxbNeoY,Yongky 1,"696G+C4G Jogja dekat rel, Bausasran, Kec. Danu...",https://streetviewpixels-pa.googleapis.com/v1/...,0,Kafe dan Kedai Kopi,"[""Kedai Kopi""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -7.7889428999999994, ""longitude"":...",Not Present
30185,ChIJF2FHPgBXei4R1LyeLo47Ppc,angkringan sekh wanto,"Jl. Tegal Panggung No.4, Tegal Panggung, Kec. ...",https://streetviewpixels-pa.googleapis.com/v1/...,0,Kafe dan Kedai Kopi,"[""Kedai Kopi""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -7.7966305, ""longitude"": 110.3714...",Not Present
30186,ChIJh2ZkJ34xEm4R7QzIh_u7Gg4,catering,NaN,NaN,0,Toko Makanan dan Minuman,"[""Toko Makanan Sehat"", ""Service establishment""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -2.44565, ""longitude"": 117.888799...",Not Present
30187,ChIJ0V0zLK9Zei4RUUy9oy4BRa8,yogyakarta,"6939+QQ9, Suryatmajan, Kec. Danurejan, Kota Yo...",https://streetviewpixels-pa.googleapis.com/v1/...,0,Lainnya,"[""Toko bahan makanan""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -7.7955798, ""longitude"": 110.3694...",Not Present


### Memfilter Coordinates Indonesia

In [31]:
# Konversi string JSON menjadi dua kolom terpisah: latitude dan longitude
full_df[['latitude', 'longitude']] = full_df['coordinates'].apply(lambda x: pd.Series(json.loads(x)))

# Filter data berdasarkan koordinat untuk memilih hanya data di dalam batas wilayah Indonesia
filtered_df = full_df[(full_df['latitude'] >= -11.00) & (full_df['latitude'] <= 6.00) & 
                      (full_df['longitude'] >= 95.00) & (full_df['longitude'] <= 141.00)]

filtered_df.drop(['latitude', 'longitude'], axis=1, inplace=True)

C:\Users\fachr\AppData\Local\Temp\ipykernel_36372\656251635.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop(['latitude', 'longitude'], axis=1, inplace=True)


In [32]:
full_df = filtered_df.copy()

In [33]:
full_df

,place_id,name,address,featured_image,reviews,main_category,categories,rating,reviews_per_rating,coordinates,most_popular_times
0,ChIJd0EvQ4_zaS4ROHLtujX-hmE,Bebek Kaleyo Tebet,"Lapangan Ros Selatan No.49, RT.4/RW.1, Tebet T...",https://lh5.googleusercontent.com/p/AF1QipO_gO...,11016,Restoran,"[""Restoran Indonesia"", ""Restoran Asia""]",4.50,"{""1"": 184, ""2"": 152, ""3"": 791, ""4"": 2528, ""5"":...","{""latitude"": -6.2258, ""longitude"": 106.8568683}","[{""hour_of_day"": 19, ""average_popularity"": 80...."
1,ChIJEwwrrXH3aS4RcBo0XDRfOnc,McDonald's Puri Kembangan,"Jl. Puri Indah Raya Blok U1 No.1, RT.3/RW.2, K...",https://lh5.googleusercontent.com/p/AF1QipNF6A...,9072,Restoran,"[""Restoran Cepat Saji""]",4.60,"{""1"": 204, ""2"": 117, ""3"": 490, ""4"": 1760, ""5"":...","{""latitude"": -6.187604599999999, ""longitude"": ...","[{""hour_of_day"": 23, ""average_popularity"": 88...."
2,ChIJC_lABffzaS4RZGzb2lD-iAw,Setiabudi One,"Jl. H. R. Rasuna Said No.Kav.62, Kuningan, Kar...",https://lh5.googleusercontent.com/p/AF1QipMPXl...,8456,Restoran,"[""Pujasera""]",4.60,"{""1"": 61, ""2"": 48, ""3"": 529, ""4"": 1996, ""5"": 5...","{""latitude"": -6.2152515, ""longitude"": 106.8299...","[{""hour_of_day"": 18, ""average_popularity"": 84...."
3,ChIJwZMcGHXxaS4RKdlMwHZXitw,KFC Gunawarman,"No.80 Blok Q1, Jl. Gunawarman, Rw. Bar., Kec. ...",https://lh5.googleusercontent.com/p/AF1QipNYUw...,5918,Restoran,"[""Restoran Cepat Saji"", ""Kafe"", ""Restoran Masa...",4.50,"{""1"": 165, ""2"": 76, ""3"": 465, ""4"": 1300, ""5"": ...","{""latitude"": -6.238975, ""longitude"": 106.8074383}","[{""hour_of_day"": 8, ""average_popularity"": 74.8..."
4,ChIJp5VZTgT0aS4R26oBEPT4nUw,Warung MJS,"Jl. Setia Budi Tengah No.11, RT.1/RW.1, Kuning...",https://lh5.googleusercontent.com/p/AF1QipO6TD...,5896,Restoran,"[""Restoran Jawa""]",4.50,"{""1"": 76, ""2"": 63, ""3"": 385, ""4"": 1478, ""5"": 3...","{""latitude"": -6.2087667, ""longitude"": 106.8267...","[{""hour_of_day"": 13, ""average_popularity"": 91...."
...,...,...,...,...,...,...,...,...,...,...,...
30184,ChIJG3hFmuRZei4Ro7gGAxbNeoY,Yongky 1,"696G+C4G Jogja dekat rel, Bausasran, Kec. Danu...",https://streetviewpixels-pa.googleapis.com/v1/...,0,Kafe dan Kedai Kopi,"[""Kedai Kopi""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -7.7889428999999994, ""longitude"":...",Not Present
30185,ChIJF2FHPgBXei4R1LyeLo47Ppc,angkringan sekh wanto,"Jl. Tegal Panggung No.4, Tegal Panggung, Kec. ...",https://streetviewpixels-pa.googleapis.com/v1/...,0,Kafe dan Kedai Kopi,"[""Kedai Kopi""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -7.7966305, ""longitude"": 110.3714...",Not Present
30186,ChIJh2ZkJ34xEm4R7QzIh_u7Gg4,catering,NaN,NaN,0,Toko Makanan dan Minuman,"[""Toko Makanan Sehat"", ""Service establishment""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -2.44565, ""longitude"": 117.888799...",Not Present
30187,ChIJ0V0zLK9Zei4RUUy9oy4BRa8,yogyakarta,"6939+QQ9, Suryatmajan, Kec. Danurejan, Kota Yo...",https://streetviewpixels-pa.googleapis.com/v1/...,0,Lainnya,"[""Toko bahan makanan""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -7.7955798, ""longitude"": 110.3694...",Not Present


## Convert JSON

### most_popular_times

In [34]:
# Fungsi untuk mendapatkan rata-rata, standar deviasi, dan waktu rata-rata dari baris most_popular_times
def extract_features(row):
    if row == None:
        return pd.Series([None, None, None])
    
    data = json.loads(row)
    hour_list = [item['hour_of_day'] for item in data]
    popularity_list = [item['average_popularity'] for item in data]

    average_hour = np.mean(hour_list)
    std_hour = np.std(hour_list)
    avg_popularity = np.mean(popularity_list)

    return pd.Series([average_hour, std_hour, avg_popularity])

# Mengganti 'Not Present' dengan None pada kolom most_popular_times
full_df['most_popular_times'] = full_df['most_popular_times'].replace('Not Present', None)

# Membuat kolom baru berdasarkan most_popular_times
new_cols = full_df['most_popular_times'].apply(extract_features)
new_cols.columns = ['average_hour', 'std_hour', 'avg_popularity']

# Menggabungkan kolom baru dengan DataFrame awal
full_df = pd.concat([full_df, new_cols], axis=1)

In [35]:
# Fungsi untuk mendapatkan nilai top_hour_popularity dan top_average_popularity
def extract_top_values(row):
    if row == None:
        return pd.Series([None, None])
    
    data = json.loads(row)
    max_popularity_item = max(data, key=lambda x: x['average_popularity'])
    top_hour_popularity = max_popularity_item['hour_of_day']
    top_average_popularity = max_popularity_item['average_popularity']
    
    return pd.Series([top_hour_popularity, top_average_popularity])

# Membuat kolom baru untuk top_hour_popularity dan top_average_popularity
top_values_cols = full_df['most_popular_times'].apply(extract_top_values)
top_values_cols.columns = ['top_hour_popularity', 'top_average_popularity']

# Menggabungkan kolom baru dengan DataFrame awal
full_df = pd.concat([full_df, top_values_cols], axis=1)


In [36]:
# Menampilkan DataFrame yang telah dimodifikasi
full_df

,place_id,name,address,featured_image,reviews,main_category,categories,rating,reviews_per_rating,coordinates,most_popular_times,average_hour,std_hour,avg_popularity,top_hour_popularity,top_average_popularity
0,ChIJd0EvQ4_zaS4ROHLtujX-hmE,Bebek Kaleyo Tebet,"Lapangan Ros Selatan No.49, RT.4/RW.1, Tebet T...",https://lh5.googleusercontent.com/p/AF1QipO_gO...,11016,Restoran,"[""Restoran Indonesia"", ""Restoran Asia""]",4.50,"{""1"": 184, ""2"": 152, ""3"": 791, ""4"": 2528, ""5"":...","{""latitude"": -6.2258, ""longitude"": 106.8568683}","[{""hour_of_day"": 19, ""average_popularity"": 80....",19.00,0.82,76.52,19.00,80.14
1,ChIJEwwrrXH3aS4RcBo0XDRfOnc,McDonald's Puri Kembangan,"Jl. Puri Indah Raya Blok U1 No.1, RT.3/RW.2, K...",https://lh5.googleusercontent.com/p/AF1QipNF6A...,9072,Restoran,"[""Restoran Cepat Saji""]",4.60,"{""1"": 204, ""2"": 117, ""3"": 490, ""4"": 1760, ""5"":...","{""latitude"": -6.187604599999999, ""longitude"": ...","[{""hour_of_day"": 23, ""average_popularity"": 88....",15.00,10.61,78.76,23.00,88.29
2,ChIJC_lABffzaS4RZGzb2lD-iAw,Setiabudi One,"Jl. H. R. Rasuna Said No.Kav.62, Kuningan, Kar...",https://lh5.googleusercontent.com/p/AF1QipMPXl...,8456,Restoran,"[""Pujasera""]",4.60,"{""1"": 61, ""2"": 48, ""3"": 529, ""4"": 1996, ""5"": 5...","{""latitude"": -6.2152515, ""longitude"": 106.8299...","[{""hour_of_day"": 18, ""average_popularity"": 84....",16.67,2.62,81.33,18.00,84.14
3,ChIJwZMcGHXxaS4RKdlMwHZXitw,KFC Gunawarman,"No.80 Blok Q1, Jl. Gunawarman, Rw. Bar., Kec. ...",https://lh5.googleusercontent.com/p/AF1QipNYUw...,5918,Restoran,"[""Restoran Cepat Saji"", ""Kafe"", ""Restoran Masa...",4.50,"{""1"": 165, ""2"": 76, ""3"": 465, ""4"": 1300, ""5"": ...","{""latitude"": -6.238975, ""longitude"": 106.8074383}","[{""hour_of_day"": 8, ""average_popularity"": 74.8...",12.00,4.97,71.86,8.00,74.86
4,ChIJp5VZTgT0aS4R26oBEPT4nUw,Warung MJS,"Jl. Setia Budi Tengah No.11, RT.1/RW.1, Kuning...",https://lh5.googleusercontent.com/p/AF1QipO6TD...,5896,Restoran,"[""Restoran Jawa""]",4.50,"{""1"": 76, ""2"": 63, ""3"": 385, ""4"": 1478, ""5"": 3...","{""latitude"": -6.2087667, ""longitude"": 106.8267...","[{""hour_of_day"": 13, ""average_popularity"": 91....",16.67,2.62,89.57,13.00,91.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30184,ChIJG3hFmuRZei4Ro7gGAxbNeoY,Yongky 1,"696G+C4G Jogja dekat rel, Bausasran, Kec. Danu...",https://streetviewpixels-pa.googleapis.com/v1/...,0,Kafe dan Kedai Kopi,"[""Kedai Kopi""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -7.7889428999999994, ""longitude"":...",None,NaN,NaN,NaN,NaN,NaN
30185,ChIJF2FHPgBXei4R1LyeLo47Ppc,angkringan sekh wanto,"Jl. Tegal Panggung No.4, Tegal Panggung, Kec. ...",https://streetviewpixels-pa.googleapis.com/v1/...,0,Kafe dan Kedai Kopi,"[""Kedai Kopi""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -7.7966305, ""longitude"": 110.3714...",None,NaN,NaN,NaN,NaN,NaN
30186,ChIJh2ZkJ34xEm4R7QzIh_u7Gg4,catering,NaN,NaN,0,Toko Makanan dan Minuman,"[""Toko Makanan Sehat"", ""Service establishment""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -2.44565, ""longitude"": 117.888799...",None,NaN,NaN,NaN,NaN,NaN
30187,ChIJ0V0zLK9Zei4RUUy9oy4BRa8,yogyakarta,"6939+QQ9, Suryatmajan, Kec. Danurejan, Kota Yo...",https://streetviewpixels-pa.googleapis.com/v1/...,0,Lainnya,"[""Toko bahan makanan""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -7.7955798, ""longitude"": 110.3694...",None,NaN,NaN,NaN,NaN,NaN


## Nearest Competitor

In [37]:
# Ekstrak latitude dan longitude dari kolom 'coordinates'
coords = full_df['coordinates'].apply(json.loads).apply(lambda x: (x['latitude'], x['longitude']))
coords = np.deg2rad(np.array(coords.tolist()))

# Buat KDTree dari koordinat
tree = KDTree(coords)

# Cari 3 tetangga terdekat untuk setiap baris (termasuk dirinya sendiri)
distances, indices = tree.query(coords, k=3)

# Memastikan kompetitor tidak sama dengan baris itu sendiri
nearest_indices = []
nearest_distances = []

for i in range(len(full_df)):
    for j in range(1, 3):  # Mulai dari 1 untuk menghindari dirinya sendiri
        if full_df['place_id'].iloc[i] != full_df['place_id'].iloc[indices[i, j]]:
            nearest_indices.append(indices[i, j])
            nearest_distances.append(distances[i, j] * 6371000)  # Konversi dari radian ke meter
            break

# Mengambil data kompetitor terdekat dari full_df menggunakan nearest_indices
competitor_place_ids = full_df['place_id'].values[nearest_indices]
nearest_top_hour_popularity = full_df['top_hour_popularity'].values[nearest_indices]
nearest_top_average_popularity = full_df['top_average_popularity'].values[nearest_indices]
competitor_ratings = full_df['rating'].values[nearest_indices]

# Menambahkan kolom informasi kompetitor terdekat ke DataFrame asli
full_df['nearest_competitor_place_id'] = competitor_place_ids
full_df['nearest_competitor_distance'] = nearest_distances
full_df['nearest_competitor_top_hour_popularity'] = nearest_top_hour_popularity
full_df['nearest_competitor_top_average_popularity'] = nearest_top_average_popularity
full_df['nearest_competitor_rating'] = competitor_ratings

In [38]:
full_df.sort_values('nearest_competitor_distance', ascending=False).head(50)

,place_id,name,address,featured_image,reviews,main_category,categories,rating,reviews_per_rating,coordinates,...,average_hour,std_hour,avg_popularity,top_hour_popularity,top_average_popularity,nearest_competitor_place_id,nearest_competitor_distance,nearest_competitor_top_hour_popularity,nearest_competitor_top_average_popularity,nearest_competitor_rating
21579,ChIJKWJ8oJ6_eS4R4u4x8Y-mU6A,Gigafood,NaN,NaN,0,Toko Makanan dan Minuman,"[""Toko Makanan Sehat"", ""Service establishment""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -0.906712, ""longitude"": 110.2204299}",...,NaN,NaN,NaN,NaN,NaN,ChIJMVFzRHgicS4RXbsPj8S7kxw,627259.03,19.00,86.86,3.90
4398,ChIJU2IFWyLtaS4RZV71_a-PvX0,MEDAN FOOD DISTRIBUTOR,NaN,https://lh5.googleusercontent.com/p/AF1QipNFDj...,0,Lainnya,"[""Paket Makanan Pokok"", ""Service establishment""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": 3.6422865, ""longitude"": 98.6694935}",...,NaN,NaN,NaN,NaN,NaN,ChIJG2Xo2dMLeC4RDZJbXBm0nss,121791.80,16.00,46.71,0.00
21634,ChIJG2Xo2dMLeC4RDZJbXBm0nss,Kosong,NaN,https://lh5.googleusercontent.com/p/AF1QipNzAf...,0,Toko Makanan dan Minuman,"[""Toko Makanan"", ""Service establishment""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": 3.3380285, ""longitude"": 97.617301}",...,16.00,0.82,42.48,16.00,46.71,ChIJU2IFWyLtaS4RZV71_a-PvX0,121791.80,NaN,NaN,0.00
2105,ChIJbVidTJanQS4RI1ovJZKy1Gc,GEMOY DRINK & FOOD,"Pematang Pasir, Kec. Ketapang, Kabupaten Lampu...",https://lh5.googleusercontent.com/p/AF1QipMYnC...,0,Kafe dan Kedai Kopi,"[""Kedai Kopi""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -5.681773499999999, ""longitude"": ...",...,NaN,NaN,NaN,NaN,NaN,ChIJvXXB0UqRQS4RfDpQbmlaobo,35639.31,NaN,NaN,5.00
24058,ChIJTR2HXhj71y0RziXDIdmxg1M,Selera Indonesia Food,NaN,https://lh5.googleusercontent.com/p/AF1QipP2ql...,0,Toko Makanan dan Minuman,"[""Toko Makanan"", ""Service establishment""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -6.914708999999999, ""longitude"": ...",...,NaN,NaN,NaN,NaN,NaN,ChIJW4aydqUjdy4RYo1tMlzQE0s,22661.63,NaN,NaN,4.40
15013,ChIJJ4nFMwh3by4RFhjJh5ddoNg,Rumah,"Banjarsari, Banjaran, Kec. Salem, Kabupaten Br...",https://streetviewpixels-pa.googleapis.com/v1/...,0,Kafe dan Kedai Kopi,"[""Kafe""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -7.1932503, ""longitude"": 108.7955...",...,NaN,NaN,NaN,NaN,NaN,ChIJZcyubcGFby4RaUPsc4wqEf0,19875.91,NaN,NaN,0.00
23241,ChIJrdmVdGLNeS4RAxjjy-d8QmM,ANGKRINGANEM,"Jl. Raya Gondang, Tikung Wetan, Gondang, Kec. ...",https://lh5.googleusercontent.com/p/AF1QipNoyW...,4,Kafe dan Kedai Kopi,"[""Kedai Kopi""]",5.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 4}","{""latitude"": -7.4000933, ""longitude"": 111.8471...",...,NaN,NaN,NaN,NaN,NaN,ChIJI9xX7PbReS4Rtat4pq0OPX4,17084.82,NaN,NaN,0.00
10313,ChIJJ-6jiSknaS4Rgt-srPW40Qw,Jasmine Food and Beverage,"kp.tanjung,RT/RW.06/03, desa, Kawungluwuk, Kec...",https://lh5.googleusercontent.com/p/AF1QipP27g...,8,Lainnya,"[""Kedai Sarapan""]",4.10,"{""1"": 1, ""2"": 0, ""3"": 0, ""4"": 3, ""5"": 4}","{""latitude"": -6.744345399999999, ""longitude"": ...",...,16.00,2.16,33.33,17.00,36.29,ChIJMcXCVgAjaS4Rz_PAa542NlM,13662.00,NaN,NaN,5.00
23937,ChIJI9xX7PbReS4Rtat4pq0OPX4,Gondang,"MP75+V43, Dusun Kleco, Nglampin, Kec. Ngambon,...",NaN,0,Bar dan Lounge,"[""Bar""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -7.335375, ""longitude"": 111.7078073}",...,NaN,NaN,NaN,NaN,NaN,ChIJNbZidZjFeS4RKBSzgxxP5hw,12742.40,NaN,NaN,0.00
19861,ChIJn2YrsTrjdy4RUEX7nA8bSWY,food and beverage,"Jl. Wisuda, RT.01/RW.01, Abar Abir, Abar-Abir,...",https://streetviewpixels-pa.googleapis.com/v1/...,0,Restoran,"[""Restoran""]",0.00,"{""1"": 0, ""2"": 0, ""3"": 0, ""4"": 0, ""5"": 0}","{""latitude"": -7.0339925, ""longitude"": 112.5692...",...,NaN,NaN,NaN,NaN,NaN,ChIJIwQl6LH_dy4RVkz6L1oEVcI,12731.84,12.00,33.00,5.00


## Export

In [39]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30188 entries, 0 to 30188
Data columns (total 21 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   place_id                                   30188 non-null  object 
 1   name                                       30178 non-null  object 
 2   address                                    30105 non-null  object 
 3   featured_image                             29326 non-null  object 
 4   reviews                                    30188 non-null  int64  
 5   main_category                              30188 non-null  object 
 6   categories                                 30188 non-null  object 
 7   rating                                     30188 non-null  float64
 8   reviews_per_rating                         30188 non-null  object 
 9   coordinates                                30188 non-null  object 
 10  most_popular_times         

In [40]:
full_df.to_csv('full_data.csv', index=False)
full_df.to_excel('full_data.xlsx', index=False)